In [1]:
""" Data Cleaning """
# << 데이터 처리에 대한 이슈 >>
# 데이터의 최대 / 최소가 다름 -> Scale 에 따른 y값에 영향
# Ordinary 또는 Nominal 한 값들의 표현은 어떻게 ?
# 잘 못 기입된 값들에 대한 처리
# 값이 없을 경우는 어떻게 ?
# 극단적으로 큰 값 또는 작은 값들은 그대로 놔둬야 하는가 ?

# << 데이터 전처리에 대한 이슈 >>
# 데이터가 빠진 경우 (결측치의 처리)
# 라벨링된 데이터(category) 데이터의 처리
# 데이터의 scale 의 차이가 매우 크게 날 경우

import pandas as pd
import numpy as np

In [2]:
""" Missing Values """
# 데이터가 없으면 sample 을 drop (좋지 않은 방법 -> 한 두개만 빌 떄 사용)
# 데이터가 없는 '최소 개수' 를 정해서 'sample 을 drop'
# 데이터가 거의 없는 feature 는 'feature 자체를 drop'
# 최빈값, 평균값, 중간값 등으로 비어있는 데이터를 채우기

# Eaxmple from - https://chrisalbon.com/python/pandas_missing_data.html
raw_data = {"first_name": ["Jason", np.nan, "Tina", "Jake", "Amy"],
            "last_name": ["Miller", np.nan, "Ali", "Milner", "Cooze"],
            "age": [42, np.nan, 36, 24, 73],
            "sex": ["m", np.nan, "f", "m", "f"],
            "preTestScore": [4, np.nan, np.nan, 2, 3],
            "postTestScore": [25, np.nan, np.nan, 62, 70]}

df = pd.DataFrame(raw_data, columns=["first_name", "last_name", "age", "sex", "preTestScore", "postTestScore"])
df

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [3]:
# NaN 을 찾고 isnull() 함수를 이용하면 data 가 얼마나 비워져 있는지 알 수 있다.
df.isnull().sum() / len(df)  # 이런 방식이면 몇 %가 비워졌는지 알 수 있다.

first_name       0.2
last_name        0.2
age              0.2
sex              0.2
preTestScore     0.4
postTestScore    0.4
dtype: float64

In [4]:
# NaN data 가 존재하는 row 를 drop 하는 방식
df_no_missing = df.dropna()
df_no_missing

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [5]:
# 모든 data 가 NaN 인 row 를 drop 하는 방식
df_cleaned = df.dropna(how="all", axis=0)
df_cleaned

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [6]:
# NaN data 가 최소 n 개 이상 있지 않으면 drop
df_drop1 = df.dropna(thresh=5, axis=0)  # default axis 값은 0 이다.
df_drop1

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [7]:
# 데이터 삭제 작업이 끝났으면, 채워넣어야 한다.
# df.fillna(k) 를 사용하는데, 0 이라든지 평균값이라든지 적당한 값을 넣을 수 있다.
# 가장 많이 넣는 값은 '평균값', '중위값', '최빈값' 이다.

df.fillna(0)

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,0,0,0.0,0,0.0,0.0
2,Tina,Ali,36.0,f,0.0,0.0
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [8]:
df["preTestScore"].mean()  # 평균값

3.0

In [9]:
df["postTestScore"].median()  # 중위값

62.0

In [10]:
df["age"].mode()  # 최빈값 (여기선 두 개 이상 중첩되는 data 가 없으므로 따로 출력되는 값은 없다.)

0    24.0
1    36.0
2    42.0
3    73.0
dtype: float64

In [11]:
df_copied = df.copy()
df_copied["preTestScore"].fillna(df_copied["preTestScore"].mean(), inplace=True)
df_copied

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,NaN,NaN,NaN,NaN,3.0,NaN
2,Tina,Ali,36.0,f,3.0,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [12]:
# 어떤 특정 col 을 기준으로, 다른 col 의 NaN 값을 채워 넣을 수 있다.

df.groupby("sex")["postTestScore"].transform("mean")
# 여기선, "sex" col 의 같은 value 를 가진 것끼리의 postTestScore 의 평균값을 출력한다.

0    43.5
1     NaN
2    70.0
3    43.5
4    70.0
Name: postTestScore, dtype: float64

In [13]:
df_copied["postTestScore"].fillna(
    df_copied.groupby("sex")["postTestScore"].transform("mean"), inplace=True)
df_copied
# 여기선, postTestScore 의 NaN 값을 sex 를 기준으로 한 postTestScore 의 mean 값을 넣는다.

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,NaN,NaN,NaN,NaN,3.0,NaN
2,Tina,Ali,36.0,f,3.0,70.0
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [14]:
df[df["age"].notnull() & df["sex"].notnull()]
# 여기서 & 가 아닌 and 로 하면 출력이 안된다. (& 는 비트 "AND" 연산자이다.)

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [15]:
""" Category Data """
# << One-Hot Encoding >>

# 이산형 데이터는 어떻게 처리해야 하는가 ? -> One-Hot Encoding
# (One-Hot Encoding : category 변수로 되어 있는 data 를 1 과 0 으로 이루어진 이진수로 반환 
# (벡터의 크기를 분산시켜서 numerical 하게 표현한다.)

# {Green, Blue, Yellow} -> 데이터 집합

# {Green} -> [1, 0, 0]
# {Blue} -> [0, 1, 0]
# {Yellow} -> [0, 0, 1]
# 과 같이 실제 데이터 set 의 크기만큼 Binary Feature 를 생성

edges = pd.DataFrame({"source": [0, 1, 2],
                      "target": [2, 2, 3],
                      "weight": [3, 4, 5],
                      "color": ["red", "green", "blue"]})

In [16]:
# Data 의 type :: {source, target, weight, color} == {int, int, int, object}
edges.dtypes

source     int64
target     int64
weight     int64
color     object
dtype: object

In [17]:
# get_dummies 를 사용하면, type 이 object 인 데이터를 one-hot encoding 시켜준다.
pd.get_dummies(edges)

,source,target,weight,color_blue,color_green,color_red
0,0,2,3,0,0,1
1,1,2,4,0,1,0
2,2,3,5,1,0,0


In [18]:
# object 데이터를 따로 끊어서 merge 시키는 방법도 있다. (필요에 따라 사용할 것)
pd.get_dummies(edges["color"])

,blue,green,red
0,0,0,1
1,0,1,0
2,1,0,0


In [19]:
# dict type 을 이용하여 category 를 효율적으로 one-hot encoding 시켜줄 수 있다.
weight_dict = {3: "M", 4: "L", 5: "XL"}
edges["weight_sign"] = edges["weight"].map(weight_dict)
edges

,source,target,weight,color,weight_sign
0,0,2,3,red,M
1,1,2,4,green,L
2,2,3,5,blue,XL


In [20]:
# << Data Binning >>

# data 가 퍼져 있을 확률이 높으므로, 구간을 나눠 줘야 한다.

# Example from - https://chrisalbon.com/python/pandas_binning_data.html
raw_data = {"regiment": ["Nighthawks", "Nighthawks", "Nighthawks", "Nighthawks", "Dragoons", "Dragoons", "Dragoons", "Dragoons", "Scouts", "Scouts", "Scouts", "Scouts"],
            "company": ["1st", "1st", "2nd", "2nd", "1st", "1st", "2nd", "2nd", "1st", "1st", "2nd", "2nd"],
            "name": ["Miller", "Jacobson", "Ali", "Milner", "Cooze", "Jacon", "Ryaner", "Sone", "Sloan", "Piger", "Riani", "Ali"],
            "preTestScore": [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
            "postTestScore": [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}

df = pd.DataFrame(raw_data, columns=["regiment", "company", "name", "preTestScore", "postTestScore"])
df

,regiment,company,name,preTestScore,postTestScore
0,Nighthawks,1st,Miller,4,25
1,Nighthawks,1st,Jacobson,24,94
2,Nighthawks,2nd,Ali,31,57
3,Nighthawks,2nd,Milner,2,62
4,Dragoons,1st,Cooze,3,70
5,Dragoons,1st,Jacon,4,25
6,Dragoons,2nd,Ryaner,24,94
7,Dragoons,2nd,Sone,31,57
8,Scouts,1st,Sloan,2,62
9,Scouts,1st,Piger,3,70


In [21]:
# binning 을 하기 위해 구간을 할당하는 list 를 만들어 준다.
# pd.cut() 을 이용하여 category 의 값을 생성 가능하다.
bins = [0, 25, 50, 75, 100]  # 0 ~ 25 : 0 / 25 ~ 50 : 1 / 50 ~ 75 : 2 / 75 ~ 100 : 3
group_names = ["Low", "Okay", "Good", "Excellent"]  # 각 구간의 이름
categories = pd.cut(df["postTestScore"], bins, labels=group_names)
categories

0           Low
1     Excellent
2          Good
3          Good
4          Good
5           Low
6     Excellent
7          Good
8          Good
9          Good
10         Good
11         Good
Name: postTestScore, dtype: category
Categories (4, object): [Low < Okay < Good < Excellent]

In [22]:
# 위와 같이 얻은 결과를 col 에 새로이 추가 할 수 있다
df["categories"] = pd.cut(df["postTestScore"], bins, labels=group_names)
df

,regiment,company,name,preTestScore,postTestScore,categories
0,Nighthawks,1st,Miller,4,25,Low
1,Nighthawks,1st,Jacobson,24,94,Excellent
2,Nighthawks,2nd,Ali,31,57,Good
3,Nighthawks,2nd,Milner,2,62,Good
4,Dragoons,1st,Cooze,3,70,Good
5,Dragoons,1st,Jacon,4,25,Low
6,Dragoons,2nd,Ryaner,24,94,Excellent
7,Dragoons,2nd,Sone,31,57,Good
8,Scouts,1st,Sloan,2,62,Good
9,Scouts,1st,Piger,3,70,Good


In [23]:
# category 의 value 들을 count 할 수 있다.
pd.value_counts(df["categories"])

Good         8
Excellent    2
Low          2
Okay         0
Name: categories, dtype: int64

In [24]:
# 전체 데이터의 one-hot encoding.
pd.get_dummies(df)

,preTestScore,postTestScore,regiment_Dragoons,regiment_Nighthawks,regiment_Scouts,company_1st,company_2nd,name_Ali,name_Cooze,name_Jacobson,...,name_Milner,name_Piger,name_Riani,name_Ryaner,name_Sloan,name_Sone,categories_Low,categories_Okay,categories_Good,categories_Excellent
0,4,25,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,24,94,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,31,57,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2,62,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,3,70,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
5,4,25,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,24,94,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
7,31,57,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
8,2,62,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
9,3,70,0,0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [25]:
""" Feature Scaling """
# 두 변수 중 하나의 값의 크기가 너무 클 때 등등 ...
# >> Feature 간의 최대-최소값의 차이를 맞춘다.

# - Min-Max Normalization
# 기존 변수의 범위를 새로운 최대-최소로 변경
# 일반적으로 0 과 1 사이의 값으로 변경함

# - Standardization (Z-Score Normalization :: 정규분포화)
# 기존 변수의 범위를 정규 분포로 변환
# 실제 Min-Max 의 값을 모를 때 활용 가능

# 보통 sklearn 을 통해 많이 변환한다.

####

# 일반적으로 Scaling 을 하면 처리 속도가 빨라진다 !!

' Feature Scaling '